In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [40]:
from llama_index.core import Settings
from llama_index.core.callbacks import CallbackManager
from langfuse.llama_index import LlamaIndexCallbackHandler

langfuse_callback_handler = LlamaIndexCallbackHandler(
    public_key="pk-lf-29339a8f-8a05-4d10-9a0a-a4718f79a53d",
    secret_key="sk-lf-d1f94511-893b-456e-973e-f57b44d50a30",
    host="https://cloud.langfuse.com"
)
Settings.callback_manager = CallbackManager([langfuse_callback_handler])

In [7]:
"""Load from pdf file"""

from llama_index.readers.file import PyMuPDFReader

root_path = "/Users/heewungsong/Experiment/Visa_Rag"

visa_pdf_file_path = f"{root_path}/Documents/E-9 Visa Guide_한국어.pdf"
visa_docs = PyMuPDFReader().load_data(file_path=visa_pdf_file_path, metadata=True)

In [25]:
from llama_index.core.schema import TextNode
import json

metadata_json_path = "../../../Documents/E-9_Visa_Guide_한국어_metadata.json"
with open(metadata_json_path, "r") as fp:
    metadata_json = json.load(fp)

visa_nodes = []
for idx, visa_doc in enumerate(visa_docs):
    node = TextNode(
        text=visa_doc.text, 
        metadata={**metadata_json[idx], **visa_doc.metadata},
        excluded_llm_metadata_keys=["page_label", "file_path", "total_pages", "file_type", "file_size", "creation_date", "last_modified_date", "source", "file_name"],
    )
    visa_nodes.append(node)

#### Save docs into Qdrant

In [23]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

client = QdrantClient(
    "localhost",
    port="6333",
    # api_key="<qdrant-api-key>", # For Qdrant Cloud, None for local instance
)
vector_store = QdrantVectorStore(client=client, collection_name="test_collection")

storage_context = StorageContext.from_defaults(vector_store=vector_store)

qdrant_index = VectorStoreIndex.from_documents(visa_docs, storage_context=storage_context)
base_index = VectorStoreIndex.from_documents(visa_docs)

INFO:httpx:HTTP Request: GET http://localhost:6333/collections/test_collection "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:6333/collections/test_collection "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/test_collection/points?wait=true "HTTP/1.1 200 OK"
HTTP Request: PUT http://localhost:6333/collections/test_collection/points?wait=true "HTTP/1.1 200 OK"


#### Load docs from Qdrant

In [9]:
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient, models

client = QdrantClient(
    "localhost",
    port="6333",
    # api_key="<qdrant-api-key>", # For Qdrant Cloud, None for local instance
)

vector_store = QdrantVectorStore(client=client, collection_name="test_collection")

qdrant_index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
base_index = VectorStoreIndex.from_documents(visa_docs)

INFO:httpx:HTTP Request: GET http://localhost:6333/collections/test_collection "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:6333/collections/test_collection "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:6333/collections/test_collection "HTTP/1.1 200 OK"


In [27]:
base_index_from_textNodes = VectorStoreIndex(visa_nodes, show_progress=True)

Generating embeddings:   0%|          | 0/21 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [28]:
qdrant_retriever = qdrant_index.as_retriever(similarity_top_k=10)
base_retriever = base_index.as_retriever(similarity_top_k=10)

base_textNodes_retriever = base_index_from_textNodes.as_retriever(similarity_top_k=10)

In [38]:
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.schema import MetadataMode

# def pretty_print_source_nodes(source_nodes):
#     for r in source_nodes:
#         display_source_node(r, source_length=100, show_source_metadata=True, metadata_mode=MetadataMode.NONE)


def pretty_print_source_nodes(source_nodes):
    for r in source_nodes:
        source = r.metadata['source']
        text = r.text.split("\n")[0]
        print(f"{source}: {text}")
        # print(f"{r.text[:50]}\n")

In [42]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-4-1106-preview", temperature=0)

In [55]:
query = "저는 한국에 살고 있는 외국인인데, 직업을 바꾸는게 가능한가요?"
query = "한국에서 건물 청소하는 일을 할 예정인데 건물 청소는 어떤 업종이고 체류자격의 비자 타입은 무엇인가요?"
query = "현재 농장에서 일하고 있는데 이 분야가 계절적 특성을 타기 때문에 다음주 부턴 일이 없는데, 다른 분야의 사업장에 일을 구하는게 가능한가요?"
query = "올해부터 E9 비자로 식당에서 일을 할 수 있는 것 맞나요?"

source_nodes1 = qdrant_retriever.retrieve(query)
pretty_print_source_nodes(source_nodes=source_nodes1)

print(f"--------------------------------------------------------------------------------------------------------------------")

source_nodes2 = base_retriever.retrieve(query)
pretty_print_source_nodes(source_nodes=source_nodes2)

print(f"--------------------------------------------------------------------------------------------------------------------")

source_nodes3 = base_textNodes_retriever.retrieve(query)
pretty_print_source_nodes(source_nodes=source_nodes3)

5: 고용허가제해당자의사증발급허용업종및체류자격약호(기호): E-9 근로자를채용할수있는
5: 육림업, 벌목업, 임업관련서비스업
1: E-9 Visa Guide - 한국어
6: 고용허가제해당자는사증발급인정서를받아야만E-9 사증을
17: 고용허가제해당자의체류자격변경허가- 사증변경
8: 유효기간1년)
16: 고용허가제농업분야외국인근로자의근무처(직장) 추가
5: 잡지및기타인쇄물출판업
9: 고용허가제해당자의재입국특례제도(구성실근로자제도)
14: 고용허가제해당자가근무처(직장)를변경할수있는조건
--------------------------------------------------------------------------------------------------------------------
5: 고용허가제해당자의사증발급허용업종및체류자격약호(기호): E-9 근로자를채용할수있는
5: 육림업, 벌목업, 임업관련서비스업
1: E-9 Visa Guide - 한국어
6: 고용허가제해당자는사증발급인정서를받아야만E-9 사증을
17: 고용허가제해당자의체류자격변경허가- 사증변경
8: 유효기간1년)
16: 고용허가제농업분야외국인근로자의근무처(직장) 추가
5: 잡지및기타인쇄물출판업
9: 고용허가제해당자의재입국특례제도(구성실근로자제도)
14: 고용허가제해당자가근무처(직장)를변경할수있는조건
--------------------------------------------------------------------------------------------------------------------
5: 고용허가제해당자의사증발급허용업종및체류자격약호(기호): E-9 근로자를채용할수있는
6: 고용허가제해당자는사증발급인정서를받아야만E-9 사증을
17: 고용허가제해당자의체류자격변경허가- 사증변경
21: 고용허가제해당자의고용변동신고
8: 고용허가제비전문취업(E-9) 자격사증발급인정서발급절차
1: E-9 Visa Guide - 한국어
9: 고용허가제해당자의재입국특례제도(구성실근로자제도)
13: 고용허가제

In [44]:
qdrant_query_engine = qdrant_index.as_query_engine()
base_textNode_engine = base_index_from_textNodes.as_query_engine()

In [54]:
def llm_query(query: str):
    langfuse_callback_handler.set_trace_params(
        name=query,
        version="1",
        tags=["Visa Guide Doc", "QdrantEngine"]
    )
    response = qdrant_query_engine.query(query)
    print(f"QdrantEngine: {response}\n\n")

    langfuse_callback_handler.set_trace_params(
        name=query,
        version="2",
        tags=["Visa Guide Doc", "BaseTextNodeEngine"]
    )
    response = base_textNode_engine.query(query)
    print(f"BaseTextNodeEngine: {response}\n\n")

In [56]:
query = "저는 한국에 살고 있는 외국인인데, 직업을 바꾸는게 가능한가요?"
query = "한국에서 건물 청소하는 일을 할 예정인데 건물 청소는 어떤 업종이고 체류자격의 비자 타입은 무엇인가요?"
query = "현재 농장에서 일하고 있는데 이 분야가 계절적 특성을 타기 때문에 다음주 부턴 일이 없는데, 다른 분야의 사업장에 일을 구하는게 가능한가요?"
query = "올해부터 E9 비자로 식당에서 일을 할 수 있는 것 맞나요?"

llm_query(query)

QdrantEngine: 네, 올해부터 E-9 비자를 가진 외국인 근로자가 특정 조건을 충족하는 식당에서 일할 수 있습니다. 해당 조건은 음식점업 외국인력 허용 시범지역에 소재한 내국인 피보험자 수 5인 이상 사업장 중 5년 이상 영업을 유지하고 있는 사업체 또는 내국인 피보험자 수 5인 미만 사업장 중 7년 이상 영업을 유지하고 있는 사업체에 한합니다. 또한, 표준직업분류상 ‘주방보조원’ 고용에 한함을 명시하고 있습니다.


BaseTextNodeEngine: 올해부터 '한식음식점업'에서 E-9 비자를 가진 근로자를 고용할 수 있습니다.




## Get Embedding Size and Distance from OpenAIEmbedding

In this code, we first create an OpenAIEmbedding model and use it to get the embedding for a given text. 

Then, we create a QdrantClient instance to interact with the Qdrant server. 

We set the size of the vector storage in Qdrant to the number of dimensions in the embedding, which is obtained from the shape of the embedding. Finally, we create a collection in Qdrant with the specified vector parameters.


Note that the distance metric is set to models.Distance.COSINE in this example, but you can choose a different distance metric depending on your needs.

In [ ]:
from qdrant_client import QdrantClient, models
from llama_index.embeddings.openai import OpenAIEmbedding

# Create an OpenAI embedding model
embedding_model = OpenAIEmbedding()

# Get the embedding for a given text
text = "Your text to get the embedding for"
embedding = embedding_model.embed_text(text)

# Create a Qdrant client
qdrant_client = QdrantClient("localhost", port=6334)

# Set the size and distance for the vector storage in Qdrant
vector_params = models.VectorParams(
    size=embedding.shape[0],  # Get the size from the shape of the embedding
    distance=models.Distance.COSINE
)

# Create a collection in Qdrant with the specified vector parameters
collection_name = "your_collection_name"
qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=vector_params
)